In [5]:
"""
Smart Customer Support AI Chatbot using LangChain and OpenAI
A complete end-to-end implementation with memory, tools, and prompt templates.
"""

import os
from typing import Optional, List
from dotenv import load_dotenv
from datetime import datetime

from langchain_openai import ChatOpenAI
from langchain_classic.memory import ConversationBufferMemory, ConversationSummaryMemory
from langchain_classic.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_classic.tools import Tool
from langchain_classic.agents import AgentExecutor, create_openai_tools_agent
from langchain_core.messages import HumanMessage, AIMessage


In [6]:
# Load environment variables from .env file
load_dotenv()

False

In [7]:
# UTILITY FUNCTIONS & TOOLS
def get_product_info(product_name: str) -> str:
    """Retrieve product information from a knowledge base."""
    products = {
        "premium_plan": {
            "price": "$29.99/month",
            "features": ["Unlimited messages", "Priority support", "Advanced analytics"],
            "description": "Our most popular plan with all premium features"
        },
        "basic_plan": {
            "price": "$9.99/month",
            "features": ["1000 messages/month", "Email support", "Basic analytics"],
            "description": "Perfect for individuals and small teams"
        },
        "enterprise_plan": {
            "price": "Custom pricing",
            "features": ["Unlimited everything", "24/7 phone support", "Custom integrations"],
            "description": "For large organizations with advanced needs"
        }
    }
    
    product = products.get(product_name.lower().replace(" ", "_"), None)
    if product:
        return f"Product: {product_name}\nPrice: {product['price']}\nFeatures: {', '.join(product['features'])}\nDescription: {product['description']}"
    return f"Sorry, I couldn't find information about {product_name}. Available products: Premium Plan, Basic Plan, Enterprise Plan"


In [8]:
def check_billing_status(account_id: str) -> str:
    """Check billing status for an account."""
    # Simulated billing data
    billing_data = {
        "acc_123": {"status": "Active", "next_billing": "2025-12-02", "amount_due": "$0"},
        "acc_456": {"status": "Past Due", "next_billing": "2025-11-02", "amount_due": "$29.99"},
        "acc_789": {"status": "Active", "next_billing": "2025-11-15", "amount_due": "$0"}
    }
    
    account = billing_data.get(account_id, None)
    if account:
        return f"Account ID: {account_id}\nStatus: {account['status']}\nNext Billing: {account['next_billing']}\nAmount Due: {account['amount_due']}"
    return f"Account {account_id} not found in the system."

In [9]:
def get_support_faq() -> str:
    """Retrieve frequently asked questions."""
    faqs = [
        "Q: How do I cancel my subscription?\nA: Go to Settings > Billing > Cancel Plan",
        "Q: What payment methods do you accept?\nA: We accept all major credit cards and PayPal",
        "Q: Is there a free trial?\nA: Yes, 14 days free for new accounts",
        "Q: Can I change my plan anytime?\nA: Yes, upgrade or downgrade anytime with no penalty"
    ]
    return "\n\n".join(faqs)


def submit_support_ticket(issue_description: str) -> str:
    """Create a support ticket."""
    ticket_id = f"TKT-{datetime.now().strftime('%Y%m%d%H%M%S')}"
    return f"Support ticket created successfully!\nTicket ID: {ticket_id}\nIssue: {issue_description}\nYou'll receive updates via email."


In [10]:
class SmartChatbot:
    """
    A customer support chatbot powered by LangChain and OpenAI.
    Features memory management, tools/functions, and customizable prompts.
    """
    
    def __init__(self, temperature: float = 0.7, use_summary_memory: bool = False):
        """
        Initialize the chatbot.
        
        Args:
            temperature: Controls randomness (0=deterministic, 1=creative)
            use_summary_memory: Use summary memory instead of buffer memory
        """
        # Validate API key
        api_key = os.getenv("OPENAI_API_KEY")
        if not api_key:
            raise ValueError(
                "OPENAI_API_KEY not found. Please create a .env file with your API key."
            )
        
        # Initialize the language model
        self.llm = ChatOpenAI(
            model_name="gpt-4o-mini",  # Fast and cost-effective
            temperature=temperature,
            api_key=api_key
        )
        
        # Initialize memory (persists conversation history)
        if use_summary_memory:
            self.memory = ConversationSummaryMemory(
                llm=self.llm,
                memory_key="chat_history",
                return_messages=True
            )
        else:
            self.memory = ConversationBufferMemory(
                memory_key="chat_history",
                return_messages=True,
                human_prefix="Customer",
                ai_prefix="Support Agent"
            )
        
        # Define tools for the agent
        self.tools = self._create_tools()
        
        # Create the system prompt
        self.prompt = self._create_prompt()
        
        # Create the agent executor
        self.agent_executor = self._create_agent()

In [11]:
def _create_tools(self) -> List[Tool]:
        """Create tools available to the agent."""
        return [
            Tool(
                name="Product_Information",
                func=get_product_info,
                description="Get detailed information about products and pricing. Input: product name"
            ),
            Tool(
                name="Check_Billing",
                func=check_billing_status,
                description="Check billing status and account information. Input: account ID"
            ),
            Tool(
                name="FAQ_Lookup",
                func=get_support_faq,
                description="Get frequently asked questions and answers. No input needed."
            ),
            Tool(
                name="Submit_Ticket",
                func=submit_support_ticket,
                description="Create a support ticket for unresolved issues. Input: issue description"
            )
        ]
    


In [12]:
def _create_prompt(self) -> ChatPromptTemplate:
        """Create a customized system prompt for the chatbot."""
        system_prompt = """You are a professional, friendly customer support agent for a SaaS platform.
        
Your responsibilities:
- Answer customer questions about products, pricing, and billing
- Help troubleshoot common issues
- Provide solutions based on our knowledge base
- Escalate complex issues by creating support tickets
- Be empathetic, clear, and concise in your responses

Guidelines:
- If you don't know something, offer to create a support ticket
- Always provide accurate information from the tools available
- Maintain a professional yet friendly tone
- Ask clarifying questions if needed
- Summarize solutions before closing conversations

Available tools: Product_Information, Check_Billing, FAQ_Lookup, Submit_Ticket
"""
        
        prompt = ChatPromptTemplate.from_messages([
            ("system", system_prompt),
            MessagesPlaceholder(variable_name="chat_history"),
            ("human", "{input}"),
            MessagesPlaceholder(variable_name="agent_scratchpad")
        ])
        
        return prompt

In [14]:
def _create_agent(self) -> AgentExecutor:
        """Create the agent with tools and memory."""
        agent = create_openai_tools_agent(
            llm=self.llm,
            tools=self.tools,
            prompt=self.prompt
        )
        
        agent_executor = AgentExecutor(
            agent=agent,
            tools=self.tools,
            memory=self.memory,
            verbose=True,  # Set to False to hide internal reasoning
            handle_parsing_errors=True,
            max_iterations=5
        )
        
        return agent_executor

In [15]:
def chat(self, user_message: str) -> str:
        """
        Send a message to the chatbot and get a response.
        
        Args:
            user_message: The user's input message
            
        Returns:
            The chatbot's response
        """
        response = self.agent_executor.invoke({
            "input": user_message,
            "chat_history": self.memory.chat_memory.messages
        })
        
        return response.get("output", "Sorry, I encountered an error processing your request.")

In [16]:
def get_conversation_history(self) -> str:
        """Retrieve the full conversation history."""
        history = []
        for message in self.memory.chat_memory.messages:
            if isinstance(message, HumanMessage):
                history.append(f"Customer: {message.content}")
            elif isinstance(message, AIMessage):
                history.append(f"Agent: {message.content}")
        
        return "\n".join(history) if history else "No conversation history yet."

In [17]:
def clear_memory(self) -> None:
        """Clear conversation history."""
        self.memory.chat_memory.clear()
        print("Conversation history cleared.")


In [18]:
# MAIN INTERACTIVE LOOP

def main():
    """Run the chatbot in interactive mode."""
    print("\n" + "="*70)
    print("🤖 SMART CUSTOMER SUPPORT CHATBOT")
    print("="*70)
    print("Type 'quit' to exit, 'history' to see conversation, or 'clear' to reset memory\n")
    
    try:
        chatbot = SmartChatbot(temperature=0.7)
        print("✅ Chatbot initialized successfully!\n")
    except ValueError as e:
        print(f"❌ Error: {e}")
        return
    
    while True:
        try:
            user_input = input("You: ").strip()
            
            if not user_input:
                continue
            
            if user_input.lower() == "quit":
                print("\nGoodbye! Thank you for using our support chatbot.")
                break
            
            if user_input.lower() == "history":
                print("\n--- Conversation History ---")
                print(chatbot.get_conversation_history())
                print("----------------------------\n")
                continue
            
            if user_input.lower() == "clear":
                chatbot.clear_memory()
                continue
            
            # Get response from chatbot
            print("\nAgent: ", end="", flush=True)
            response = chatbot.chat(user_input)
            print(response)
            print()
        
        except KeyboardInterrupt:
            print("\n\nChatbot terminated by user.")
            break
        except Exception as e:
            print(f"\n❌ Error: {e}\n")


if __name__ == "__main__":
    main()


🤖 SMART CUSTOMER SUPPORT CHATBOT
Type 'quit' to exit, 'history' to see conversation, or 'clear' to reset memory

❌ Error: OPENAI_API_KEY not found. Please create a .env file with your API key.
